# New nlpfuncs for hathi/digitizedwork

In [ ]:
%run ppa_init.ipynb
%run ppa_nlpfuncs.ipynb

## Querying for matches

In [40]:
class WordQuery:
    
    def __init__(self,word='',words=[]):
        if word: words+=[word]
        self._words = words
        self._works = []
        self._counts_df = None
        self._qset = None
    
    def get_queryset(self):
        if self._qset is None:
            self._qset=WorkWordCounts.objects.filter(data__has_any_keys=self.get_words())
        return self._qset

    def get_works(self):
        return [wwc.work for wwc in self.get_queryset()]
    
    def get_words(self):
        return self._words
    
    def get_counts(self):
        o = []
        iterr = self.get_queryset()
        for wwc in tqdm(iterr):
            work = wwc.work
            countd = wwc.data
            total = sum(countd.values())
            for word in self.get_words():
                count = countd.get(word,0)
                odx = dict(
                    work=work.id,
                    word=word,
                    count=count,
                    total=total,
                    fpm=count/total*10**6
                )
                o.append(odx)
        self._counts_df = pd.DataFrame(o).sort_values('fpm',ascending=False)        
        return self._counts_df
    
    def get_meta(self, keys = ['title','author','pub_date']):
        o = []
        for work in self.get_works():
            d = work.__dict__
            odx = {k:d.get(k,'') for k in keys}
            o.append(dict(work=work.id, **odx))
        return pd.DataFrame(o)
    

In [41]:
# work.__dict__

In [42]:
wq = WordQuery('ballad')
wq.get_queryset()

<QuerySet [<WorkWordCounts: WorkWordCounts object (1)>, <WorkWordCounts: WorkWordCounts object (3)>, <WorkWordCounts: WorkWordCounts object (4)>, <WorkWordCounts: WorkWordCounts object (5)>, <WorkWordCounts: WorkWordCounts object (8)>, <WorkWordCounts: WorkWordCounts object (10)>, <WorkWordCounts: WorkWordCounts object (13)>, <WorkWordCounts: WorkWordCounts object (14)>, <WorkWordCounts: WorkWordCounts object (15)>, <WorkWordCounts: WorkWordCounts object (16)>, <WorkWordCounts: WorkWordCounts object (17)>, <WorkWordCounts: WorkWordCounts object (19)>, <WorkWordCounts: WorkWordCounts object (21)>, <WorkWordCounts: WorkWordCounts object (22)>, <WorkWordCounts: WorkWordCounts object (23)>, <WorkWordCounts: WorkWordCounts object (24)>, <WorkWordCounts: WorkWordCounts object (25)>, <WorkWordCounts: WorkWordCounts object (26)>, <WorkWordCounts: WorkWordCounts object (27)>, <WorkWordCounts: WorkWordCounts object (28)>, '...(remaining elements truncated)...']>

In [43]:
wq.get_meta()w

,work,title,author,pub_date
0,1133,A course in expository writing.,"Buck, Gertrude, 1871-1922",1901
1,4593,A manual of composition and rhetoric:,"Hart, John S. (John Seely), 1810-1877",1892
2,4665,Imagination and fancy;,"Hunt, Leigh, 1784-1859",1871
3,1996,"Periods of European literature,",,1897
4,2058,On early English pronunciation :,"Ellis, Alexander John, 1814-1890",1869
...,...,...,...,...
2594,2911,The Writer's handbook;,,1888
2595,931,"The younger American poets,","Rittenhouse, Jessie B. (Jessie Belle), 1869-1948",1904
2596,3984,Zur Lautlehre der französischen Elemente in de...,"Lenz, Karl, b. 1871",1913
2597,3728,The young ladies' elocutionary reader :,"Russell, Anna U., d. 1894",1853


In [44]:
wq.get_counts()

100%|██████████| 2599/2599 [00:00<00:00, 4797.95it/s]


,work,word,count,total,fpm
1861,4999,ballad,753,97965,7686.418619
839,653,ballad,606,85770,7065.407485
1507,1948,ballad,271,52515,5160.430353
1019,827,ballad,513,131049,3914.566307
1020,909,ballad,517,132340,3906.604201
...,...,...,...,...,...
2260,615,ballad,1,569755,1.755140
87,3051,ballad,1,631234,1.584199
1554,3161,ballad,1,752117,1.329580
93,3230,ballad,1,887763,1.126427


In [ ]:
wq.get_counts().merge(wq.get_meta(), on='work', how='left')